In [1]:
import itertools

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from hyperas import optim
from tensorflow.keras.models import Model
from hyperopt import Trials, STATUS_OK, tpe
from sklearn.metrics import confusion_matrix
from hyperas.distributions import choice, uniform
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, GlobalMaxPooling2D

Using TensorFlow backend.


In [2]:
def get_data():  
  cifar10 = tf.keras.datasets.cifar10

  (x_train, y_train), (x_test, y_test) = cifar10.load_data()
  x_train, x_test = x_train / 255.0, x_test / 255.0
  y_train, y_test = y_train.flatten(), y_test.flatten()
  return x_train, y_train, x_test, y_test

K = 10

In [3]:
def create_model():
  i = Input(shape = x_train[0].shape)

  x = Conv2D(32, (3, 3), activation = 'relu', padding = 'same')(i)
  x = BatchNormalization()(x)
  x = Conv2D(32, (3, 3), activation = 'relu', padding = 'same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  x = Dropout({{uniform(0, 1)}})(x)
  x = Conv2D(64, (3, 3), activation = 'relu', padding = 'same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(64, (3, 3), activation = 'relu', padding = 'same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  x = Dropout({{uniform(0, 1)}})(x)
  x = Conv2D(128, (3, 3), activation = 'relu', padding = 'same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(128, (3, 3), activation = 'relu', padding = 'same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  x = Dropout({{uniform(0, 1)}})(x)

  if {{choice(['three', 'four'])}} == 'four':
    x = Conv2D(128, (3, 3), activation = 'relu', padding = 'same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout({{uniform(0, 1)}})(x)

  x = GlobalMaxPooling2D()(x)

  x = model.add({{choice([Dropout(0.5)(x), Activation('linear')(x)])}})
  x = Dense(1024, activation = 'relu')(x)
  x = Dropout({{uniform(0, 1)}})(x)
  x = Dense(K, activation = 'softmax')(x)

  model = Model(i, x)

  model.compile(optimizer = {{choice(['rmsprop', 'adam', 'sgd'])}},
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

  r = model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 50)

  validation_acc = np.amax(result.history['val_accuracy'])
  print('Best validation acc of epoch:', validation_acc)
  return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model} 